In [1]:
!pip install sodapy
!pip install Unidecode

In [0]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime, timedelta, date
from unidecode import unidecode
import copy

## Morbilidades


In [0]:
def read_csv_url(url):
  df = pd.read_csv(url)
  return df

In [4]:
df_morbilidades_crudo = read_csv_url("https://raw.githubusercontent.com/MiguelMque/covid19/master/tarea14/data/MORBILIDAD.csv?token=AHPH4XOMFDMUFSMKIEQORS264K32E")
df_morbilidades_crudo = df_morbilidades_crudo.rename(
    columns={
        "Gran causa de morbilidad": "causa_morbilidad",
        "Subgrupo de causas": "morbilidad",
        "Sexo": "sexo",
        "Núm.  Personas 2012": "cantidad_morbilidad"
    }
)
df_morbilidades_crudo = df_morbilidades_crudo.drop(columns=['id_depto', 'causa_morbilidad', 'sexo'])
df_morbilidades_crudo["nom_depto"] = df_morbilidades_crudo["nom_depto"].str.lower()
df_morbilidades_crudo["nom_depto"] = df_morbilidades_crudo["nom_depto"].transform(lambda x: unidecode(x))
df_morbilidades_crudo["nom_depto"] = df_morbilidades_crudo["nom_depto"].transform(lambda x: x if x != "archipielago de san andres, providencia y santa catalina" else "san andres y providencia")
df_morbilidades_crudo["nom_depto"] = df_morbilidades_crudo["nom_depto"].transform(lambda x: x if x != "bogota d.c" else "bogota")
df_morbilidades_crudo = df_morbilidades_crudo.dropna(subset=['morbilidad'])
df_morbilidades_crudo = df_morbilidades_crudo.sort_values(by=["nom_depto", "morbilidad"]).reset_index(drop=True)
df_morbilidades_crudo["morbilidad"] = df_morbilidades_crudo["morbilidad"].str.lower()
df_morbilidades_crudo["morbilidad"] = df_morbilidades_crudo["morbilidad"].transform(lambda x: unidecode(x))
df_morbilidades_crudo["morbilidad"] = df_morbilidades_crudo["morbilidad"].transform(lambda x: x.replace(' ', '_'))
df_morbilidades_crudo["morbilidad"] = df_morbilidades_crudo["morbilidad"].transform(lambda x: x.replace(',', '_'))
df_morbilidades_crudo["morbilidad"] = df_morbilidades_crudo["morbilidad"].transform(lambda x: x.replace('-', '_'))
df_morbilidades_crudo.head(1)

,nom_depto,morbilidad,cantidad_morbilidad
0,amazonas,anomalias_congenitas,44


In [5]:
# agrupar por morbilidad sumando cantidad_morbilidad
df_morbilidades_departamento = df_morbilidades_crudo.groupby(["morbilidad", "nom_depto"])["cantidad_morbilidad"].sum().reset_index()
df_morbilidades_departamento = df_morbilidades_departamento.sort_values(by=["nom_depto", "morbilidad"]).reset_index(drop=True)
df_morbilidades_departamento.head(1)

,morbilidad,nom_depto,cantidad_morbilidad
0,anomalias_congenitas,amazonas,328


In [6]:
# morbilidades queden como columnas
df_morbilidades = df_morbilidades_departamento.pivot(index="nom_depto", columns="morbilidad", values="cantidad_morbilidad")
df_morbilidades['departamento'] = df_morbilidades.index
df_morbilidades = df_morbilidades.reset_index(drop=True)
df_morbilidades.head(1)

morbilidad,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas,departamento
0,328.0,11292.0,68.0,828.0,1680.0,448.0,2132.0,704.0,2636.0,2396.0,2408.0,2420.0,4320.0,5628.0,4252.0,1820.0,4476.0,NaN,4.0,116.0,332.0,516.0,7102.0,2540.0,amazonas


## Cargando datasets variables constantes




In [7]:
df_densidad_poblacional = read_csv_url("https://raw.githubusercontent.com/MiguelMque/covid19/master/tarea14/data/densidad_poblacional_x_departamento.csv?token=AHPH4XOSOBTRI3E7CQY4Z6S63VLSC")
df_densidad_poblacional["departamento"] = df_densidad_poblacional["departamento"].str.lower()
df_densidad_poblacional["departamento"] = df_densidad_poblacional["departamento"].transform(lambda x: unidecode(x))
df_densidad_poblacional["departamento"] = df_densidad_poblacional["departamento"].transform(lambda x: x if x != "bogota d.c" else "bogota")
df_densidad_poblacional = df_densidad_poblacional.drop(columns=['superficie(km2)'])
df_densidad_poblacional.head(1)

,departamento,poblacion,personas por km2
0,amazonas,78830,0.719


In [8]:
df_edad_promedio = read_csv_url("https://raw.githubusercontent.com/MiguelMque/covid19/master/tarea14/data/edad_promedio.csv?token=AHPH4XKJCUZJD6AR6D4GJFC63VLUS")
df_edad_promedio["departamento"] = df_edad_promedio["departamento"].transform(lambda x: unidecode(x))
df_edad_promedio.head(1)

,departamento,cantidad_casos,edad_promedio
0,antioquia,5974788,34.628


In [9]:
df_indice_pobreza_municipio = read_csv_url("https://raw.githubusercontent.com/MiguelMque/covid19/master/tarea14/data/indice_pobreza.csv?token=AHPH4XLDM5RXEEJ5GBV2CKC63VLWI")
df_indice_pobreza = (
    df_indice_pobreza_municipio.groupby("Codigo Departamento")["Total "]
    .sum()
    .reset_index()
    .rename(columns={"Codigo Departamento": "codigo_departamento", "Total ": "total"})
)
df_indice_pobreza.head(1)

,codigo_departamento,total
0,5,4698.3


In [10]:
df_mayores_65 = read_csv_url("https://raw.githubusercontent.com/MiguelMque/covid19/master/tarea14/data/mayores_65.csv?token=AHPH4XPVTAUMC43QW5HKQAK63VLXM")
df_mayores_65.head(1)

,Código,Nombre del Departamento,Total
0,5,ANTIOQUIA,576614


## Uniendo datasets variables constantes




In [11]:
# Merge df_indice_pobreza con df_mayores_65
df_pobreza_65 = df_indice_pobreza.merge(
    df_mayores_65, how="outer", left_on="codigo_departamento", right_on="Código",
)
df_pobreza_65 = df_pobreza_65[["Nombre del Departamento", "Total", "total"]]
df_pobreza_65 = df_pobreza_65.rename(
    columns={
        "Nombre del Departamento": "departamento",
        "Total": "cantidad_mayores_65",
        "total": "ipm",
    }
)
df_pobreza_65["departamento"] = df_pobreza_65["departamento"].str.lower()
df_pobreza_65["departamento"] = df_pobreza_65["departamento"].transform(
    lambda x: unidecode(x)
)
df_pobreza_65["departamento"] = df_pobreza_65["departamento"].transform(
    lambda x: unidecode(x)
    if x != "san andres, providencia y santa catalina"
    else "san andres y providencia"
)
df_pobreza_65 = df_pobreza_65.sort_values(by=["departamento"])
df_pobreza_65.head(1)

,departamento,cantidad_mayores_65,ipm
28,amazonas,2791,898.6


In [12]:
# Merge df_densidad_poblacional con df_edad_promedio
df_densidad_edad = df_densidad_poblacional.merge(
    df_edad_promedio, how="outer", left_on="departamento", right_on="departamento"
)
df_densidad_edad = df_densidad_edad.sort_values(by=["departamento"])
df_densidad_edad = df_densidad_edad[
    [
        "departamento",
        "poblacion",
        "personas por km2",
        "edad_promedio",
    ]
]
df_densidad_edad = df_densidad_edad.rename(
    columns={
        "poblacion": "poblacion_total",
        "personas por km2": "personas_km2",
    }
)
df_densidad_edad = df_densidad_edad.drop([33])
df_densidad_edad.head(1)

,departamento,poblacion_total,personas_km2,edad_promedio
0,amazonas,78830,0.719,25.42


In [13]:
# Merge df_pobreza_65 con df_densidad_edad
df_variables_constantes_1 = df_pobreza_65.merge(
    df_densidad_edad, how="outer", left_on="departamento", right_on="departamento"
)
# En INS no esta guania, guaviare, vichada
df_variables_constantes_1 = df_variables_constantes_1.drop([15, 16, 32])
df_variables_constantes_1.head(1)

,departamento,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio
0,amazonas,2791,898.6,78830,0.719,25.42


In [14]:
# Merge variables constantes con morbilidades
df_variables_constantes = df_variables_constantes_1.merge(
    df_morbilidades, how="inner", left_on="departamento", right_on="departamento"
)
df_variables_constantes.head(1)

,departamento,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
0,amazonas,2791,898.6,78830,0.719,25.42,328.0,11292.0,68.0,828.0,1680.0,448.0,2132.0,704.0,2636.0,2396.0,2408.0,2420.0,4320.0,5628.0,4252.0,1820.0,4476.0,NaN,4.0,116.0,332.0,516.0,7102.0,2540.0


In [15]:
# Dataframe variables constantes por dia
df_variables_constantes_dia = pd.DataFrame(columns=["fecha"]+df_variables_constantes.columns.to_list())
df_fecha = pd.date_range(start=datetime(2020, 5, 10), end = datetime.now()-timedelta(days=2)).to_frame(index=False, name="fecha")
for i, row_i in df_fecha.iterrows():
  for j, row_j in df_variables_constantes.iterrows():
    values = {
        "fecha": datetime.strftime(row_i.fecha, '%Y-%m-%d'), 
        "departamento": row_j.departamento,
        "cantidad_mayores_65": row_j.cantidad_mayores_65,
        "ipm": row_j.ipm,
        "poblacion_total": row_j.poblacion_total,
        "personas_km2": row_j.personas_km2,
        "edad_promedio": row_j.edad_promedio,
        'anomalias_congenitas': row_j.anomalias_congenitas,
        'codiciones_orales': row_j.codiciones_orales,
        'condiciones_derivadas_durante_el_periodo_perinatal': row_j.condiciones_derivadas_durante_el_periodo_perinatal,
        'condiciones_maternas': row_j.condiciones_maternas,
        'condiciones_neuropsiquiatricas': row_j.condiciones_neuropsiquiatricas,
        'deficiencias_nutricionales': row_j.deficiencias_nutricionales,
        'desordenes_endocrinos': row_j.desordenes_endocrinos,
        'diabetes_mellitus': row_j.diabetes_mellitus,
        'enfermedades_cardiovasculares': row_j.enfermedades_cardiovasculares,
        'enfermedades_de_la_piel': row_j.enfermedades_de_la_piel,
        'enfermedades_de_los_organos_de_los_sentidos': row_j.enfermedades_de_los_organos_de_los_sentidos,
        'enfermedades_digestivas': row_j.enfermedades_digestivas,
        'enfermedades_genitourinarias': row_j.enfermedades_genitourinarias,
        'enfermedades_infecciosas_y_parasitarias': row_j.enfermedades_infecciosas_y_parasitarias,
        'enfermedades_musculo_esqueleticas': row_j.enfermedades_musculo_esqueleticas,
        'enfermedades_respiratorias': row_j.enfermedades_respiratorias,
        'infecciones_respiratorias': row_j.infecciones_respiratorias,
        'lesiones_de_intensionalidad_indeterminada': row_j.lesiones_de_intensionalidad_indeterminada,
        'lesiones_intensionales': row_j.lesiones_intensionales,
        'lesiones_no_intensionales': row_j.lesiones_no_intensionales,
        'neoplasias_malignas': row_j.neoplasias_malignas,
        'otras_neoplasias': row_j.otras_neoplasias,
        'signos_y_sintomas_mal_definidos': row_j.signos_y_sintomas_mal_definidos,
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas': row_j.traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
    }
    df_variables_constantes_dia = df_variables_constantes_dia.append(values, ignore_index=True)
df_variables_constantes_dia[df_variables_constantes_dia['departamento'] == 'cordoba'].tail(3)

,fecha,departamento,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
673,2020-06-01,cordoba,130407,1603.5,1788648,714.887,31.305,13674.0,311828.0,5970.0,54860.0,108318.0,75406.0,123286.0,38968.0,217360.0,168458.0,153818.0,190854.0,410046.0,290464.0,283604.0,108388.0,378222.0,84.0,298.0,47860.0,12154.0,33516.0,481762.0,145144.0
703,2020-06-02,cordoba,130407,1603.5,1788648,714.887,31.305,13674.0,311828.0,5970.0,54860.0,108318.0,75406.0,123286.0,38968.0,217360.0,168458.0,153818.0,190854.0,410046.0,290464.0,283604.0,108388.0,378222.0,84.0,298.0,47860.0,12154.0,33516.0,481762.0,145144.0
733,2020-06-03,cordoba,130407,1603.5,1788648,714.887,31.305,13674.0,311828.0,5970.0,54860.0,108318.0,75406.0,123286.0,38968.0,217360.0,168458.0,153818.0,190854.0,410046.0,290464.0,283604.0,108388.0,378222.0,84.0,298.0,47860.0,12154.0,33516.0,481762.0,145144.0


## Datos Infectados, Recuperados y Decesos

In [16]:
client = Socrata("www.datos.gov.co", None)
results = client.get("gt2j-8ykr", limit=100000)
df_casos = pd.DataFrame.from_records(results)
df_casos = df_casos.fillna("-   -")

# Transformacion campos tipo fecha
df_casos["fecha_diagnostico"] = df_casos["fecha_diagnostico"].transform(lambda x: x[:10] if x != "-   -" else x)
df_casos["fis"] = df_casos["fis"].transform(lambda x: x[:10] if x != "-   -" else x)
df_casos["fecha_de_muerte"] = df_casos["fecha_de_muerte"].transform(lambda x: x[:10] if x != "-   -" else x)
df_casos["fecha_de_notificaci_n"] = df_casos["fecha_de_notificaci_n"].transform(lambda x: x[:10] if x != "-   -" else x)
df_casos["fecha_recuperado"] = df_casos["fecha_recuperado"].transform(lambda x: x[:10] if x != "-   -" else x)
df_casos["fecha_reporte_web"] = df_casos["fecha_reporte_web"].transform(lambda x: x[:10] if x != "-   -" else x)

# Tranformacion nombres departamentos
df_casos["departamento"] = df_casos["departamento"].str.lower()
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: unidecode(x))
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if x != "bogota d.c." else "bogota")
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if "archipielago" not in x else "san andres y providencia")
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if x != "cartagena d.t. y c." else "bolivar")
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if x != "barranquilla d.e." else "atlantico")
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if x != "buenaventura d.e." else "valle del cauca")
df_casos["departamento"] = df_casos["departamento"].transform(lambda x: x if x != "santa marta d.t. y c." else "magdalena")

df_casos.head(1) #.columns.to_list()

,id_de_caso,fecha_de_notificaci_n,codigo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,fecha_de_muerte
0,1,2020-03-02,11001,Bogotá D.C.,bogota,Recuperado,19,F,Importado,Leve,Italia,2020-02-27,2020-03-06,2020-03-13,2020-03-06,PCR,- -


In [17]:
# Cantidad de infectados en total
df_casos.shape[0]

35120

In [18]:
# Cantidad de fallecidos en total
df_casos[df_casos["fecha_de_muerte"] != '-   -']["fecha_de_muerte"].shape[0]

1121

In [19]:
# Cantidad de recuperados en total
df_casos[df_casos["fecha_recuperado"] != '-   -']["fecha_recuperado"].shape[0]

12923

In [20]:
# Total x departamento
df_cuenta_departamento = (
    df_casos.groupby("departamento")["id_de_caso"]
    .count()
    .reset_index()
    .rename(columns={"id_de_caso": "infectados"})
)
df_fallecidos_departamento = (
    df_casos[df_casos["fecha_de_muerte"] != "-   -"]
    .groupby("departamento")["fecha_de_muerte"]
    .count()
    .reset_index()
    .rename(columns={"fecha_de_muerte": "decesos"})
)
df_recuperados_departamento = (
    df_casos[df_casos["fecha_recuperado"] != "-   -"]
    .groupby("departamento")["fecha_recuperado"]
    .count()
    .reset_index()
    .rename(columns={"fecha_recuperado": "recuperados"})
)

# Merge de los 3 dfs
df_cuenta_departamento = df_cuenta_departamento.merge(
    df_fallecidos_departamento,
    how="outer",
    left_on="departamento",
    right_on="departamento",
)
df_cuenta_departamento = df_cuenta_departamento.merge(
    df_recuperados_departamento,
    how="outer",
    left_on="departamento",
    right_on="departamento",
)
df_cuenta_departamento = df_cuenta_departamento.fillna(0)
df_cuenta_departamento.head(1)

,departamento,infectados,decesos,recuperados
0,amazonas,1900,69.0,803.0


In [21]:
# Merge dataset variables constantes con total I, R, D
df_ird_constantes = df_cuenta_departamento.merge(
    df_variables_constantes,
    how="outer",
    left_on="departamento",
    right_on="departamento",
)
# En INS no esta guania, guaviare, vichada
df_ird_constantes = df_ird_constantes.dropna()
df_ird_constantes = df_ird_constantes.sort_values(by=["departamento"])
df_ird_constantes.head(3)

,departamento,infectados,decesos,recuperados,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
1,antioquia,1328,6.0,533.0,576614.0,4698.3,6690977.0,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
2,arauca,2,0.0,1.0,13880.0,311.2,270708.0,11.366,29.092,2036.0,34944.0,1060.0,5640.0,10506.0,2932.0,9348.0,3274.0,17604.0,12780.0,22234.0,15142.0,32520.0,25708.0,21556.0,8108.0,24854.0,16.0,100.0,1074.0,1268.0,4222.0,48664.0,15424.0
3,atlantico,5403,146.0,1379.0,196909.0,858.5,2546138.0,751.517,32.435,39482.0,490292.0,13216.0,92378.0,276326.0,63446.0,269384.0,94614.0,528492.0,379022.0,426452.0,413394.0,669408.0,598656.0,648180.0,261712.0,620138.0,668.0,1314.0,25066.0,27472.0,72974.0,1032222.0,312074.0


In [22]:
# Total x dia x departamento
df_cuenta_departamento_dia = (
    df_casos.groupby(["fecha_reporte_web", "departamento"])["id_de_caso"]
    .count()
    .reset_index()
    .rename(columns={"id_de_caso": "infectados"})
)
df_fallecidos_departamento_dia = (
    df_casos[df_casos["fecha_de_muerte"] != "-   -"]
    .groupby(["fecha_reporte_web", "departamento"])["fecha_de_muerte"]
    .count()
    .reset_index()
    .rename(columns={"fecha_de_muerte": "decesos"})
)
df_recuperados_departamento_dia = (
    df_casos[df_casos["fecha_recuperado"] != "-   -"]
    .groupby(["fecha_reporte_web", "departamento"])["fecha_recuperado"]
    .count()
    .reset_index()
    .rename(columns={"fecha_recuperado": "recuperados"})
)

df_cuenta_departamento_dia = df_cuenta_departamento_dia.merge(
    df_fallecidos_departamento_dia,
    how="outer",
    left_on=["fecha_reporte_web", "departamento"],
    right_on=["fecha_reporte_web", "departamento"],
)
df_cuenta_departamento_dia = df_cuenta_departamento_dia.merge(
    df_recuperados_departamento_dia,
    how="outer",
    left_on=["fecha_reporte_web", "departamento"],
    right_on=["fecha_reporte_web", "departamento"],
)
df_cuenta_departamento_dia = df_cuenta_departamento_dia.fillna(0)
df_cuenta_departamento_dia.groupby(["fecha_reporte_web"])["departamento"].count().tail(30)

fecha_reporte_web
2020-05-06    11
2020-05-07    13
2020-05-08    18
2020-05-09    14
2020-05-10    24
2020-05-11    14
2020-05-12    24
2020-05-13    20
2020-05-14    15
2020-05-15    19
2020-05-16    21
2020-05-17    19
2020-05-18    18
2020-05-19    15
2020-05-20    22
2020-05-21    17
2020-05-22    20
2020-05-23    22
2020-05-24    19
2020-05-25    23
2020-05-26    21
2020-05-27    23
2020-05-28    23
2020-05-29    19
2020-05-30    19
2020-05-31    23
2020-06-01    20
2020-06-02    22
2020-06-03    22
2020-06-04    22
Name: departamento, dtype: int64

In [23]:
# Total x dia x departamento + variables constantes
df_ird_constantes_dia = df_cuenta_departamento_dia.merge(
    df_variables_constantes_dia,
    how="outer",
    left_on=["fecha_reporte_web", "departamento"],
    right_on=["fecha", "departamento"],
)
df_ird_constantes_dia = df_ird_constantes_dia[
    [
        "fecha",
        "departamento",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_ird_constantes_dia = df_ird_constantes_dia[df_ird_constantes_dia['fecha'].notna()]
df_ird_constantes_dia = df_ird_constantes_dia.fillna(0)
df_ird_constantes_dia = df_ird_constantes_dia.sort_values(by=['fecha', 'departamento']).reset_index(drop=True)
df_ird_constantes_dia.tail(1)

,fecha,departamento,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
749,2020-06-03,vaupes,0.0,0.0,0.0,1431,532.6,44928,0.83,22.849,116.0,2496.0,8.0,184.0,312.0,168.0,228.0,100.0,432.0,696.0,400.0,604.0,792.0,1648.0,912.0,208.0,948.0,0.0,0.0,12.0,48.0,72.0,1248.0,660.0


In [24]:
# Total x dia x departamento (acumulado)
df_cuenta_departamento_acum = copy.deepcopy(df_cuenta_departamento_dia)
df_cuenta_departamento_acum["infectados"] = df_cuenta_departamento_dia.groupby(["departamento"])["infectados"].cumsum()
df_cuenta_departamento_acum["decesos"] = df_cuenta_departamento_dia.groupby(["departamento"])["decesos"].cumsum()
df_cuenta_departamento_acum["recuperados"] = df_cuenta_departamento_dia.groupby(["departamento"])["recuperados"].cumsum()
df_cuenta_departamento_acum.tail(1)

,fecha_reporte_web,departamento,infectados,decesos,recuperados
1281,2020-06-04,valle del cauca,3996,193.0,1370.0


In [25]:
# Total x dia x departamento (acumulado) + variables constantes
df_ird_constantes_acum = df_cuenta_departamento_acum.merge(
    df_variables_constantes_dia,
    how="outer",
    left_on=["fecha_reporte_web", "departamento"],
    right_on=["fecha", "departamento"],
)
df_ird_constantes_acum = df_ird_constantes_acum[
    [
        "fecha",
        "departamento",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_ird_constantes_acum = df_ird_constantes_acum[df_ird_constantes_acum['fecha'].notna()]
df_ird_constantes_acum = df_ird_constantes_acum.sort_values(by=['fecha', 'departamento']).reset_index(drop=True)

# Reemplazar dias que no salieron nuevos casos con el dia anterior
for depar in df_ird_constantes_acum['departamento'].unique():
  df_ird_constantes_acum[df_ird_constantes_acum['departamento'] == depar] = df_ird_constantes_acum[df_ird_constantes_acum['departamento'] == depar].ffill(axis=0)
  df_ird_constantes_acum[df_ird_constantes_acum['departamento'] == depar] = df_ird_constantes_acum[df_ird_constantes_acum['departamento'] == depar].bfill(axis=0)

df_ird_constantes_acum.tail(1)

,fecha,departamento,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
749,2020-06-03,vaupes,11.0,11.0,0.0,1431,532.6,44928,0.83,22.849,116.0,2496.0,8.0,184.0,312.0,168.0,228.0,100.0,432.0,696.0,400.0,604.0,792.0,1648.0,912.0,208.0,948.0,NaN,NaN,12.0,48.0,72.0,1248.0,660.0


## Datos Expuestos (personas que se hicieron la prueba)



In [26]:
client = Socrata("www.datos.gov.co", None)
results = client.get("8835-5baf", limit=100000)
df_expuestos_crudos = pd.DataFrame.from_records(results)
df_expuestos_crudos = df_expuestos_crudos.dropna().reset_index(drop=True)
df_expuestos_crudos["fecha"] = df_expuestos_crudos["fecha"].transform(lambda x: x[:10])
df_expuestos_crudos = df_expuestos_crudos.drop(['acumuladas', 'procedencia_desconocida', 'guainia', 'guaviare', 'vichada'], axis=1)
df_expuestos_crudos.tail(1)

,fecha,amazonas,antioquia,arauca,atlantico,bogota,bolivar,boyaca,caldas,caqueta,casanare,cauca,cesar,choco,cordoba,cundinamarca,guajira,huila,magdalena,meta,narino,norte_de_santander,putumayo,quindio,risaralda,san_andres,santander,sucre,tolima,valle_del_cauca,vaupes
25,2020-06-04,7190,45759,1337,27344,113356,24515,8691,5066,1039,1902,3088,4525,2522,6187,12253,2033,7839,9264,9499,5992,7179,519,2544,6678,592,10208,2652,5804,32936,788


In [27]:
# Cantidad de expuestos por dia (acumulado)
df_expuestos_acum = df_expuestos_crudos.melt(id_vars=['fecha'], value_vars=df_expuestos_crudos.columns.to_list()[1:], var_name='departamento', value_name='expuestos')
df_expuestos_acum = df_expuestos_acum.sort_values(by=['fecha', 'departamento'])
df_expuestos_acum["expuestos"] = df_expuestos_acum["expuestos"].astype(float)
df_expuestos_acum["departamento"] = df_expuestos_acum["departamento"].transform(lambda x: x if x != "guajira" else "la guajira")
df_expuestos_acum["departamento"] = df_expuestos_acum["departamento"].transform(lambda x: x if x != "norte_de_santander" else "norte de santander")
df_expuestos_acum["departamento"] = df_expuestos_acum["departamento"].transform(lambda x: x if x != "san_andres" else "san andres y providencia")
df_expuestos_acum["departamento"] = df_expuestos_acum["departamento"].transform(lambda x: x if x != "valle_del_cauca" else "valle del cauca")
df_expuestos_acum.tail(3)

,fecha,departamento,expuestos
727,2020-06-04,tolima,5804.0
753,2020-06-04,valle del cauca,32936.0
779,2020-06-04,vaupes,788.0


In [28]:
# Cantidad de expuestos por dia
df_expuestos_dia = copy.deepcopy(df_expuestos_acum)
df_expuestos_primer_dia = copy.deepcopy(df_expuestos_dia.head(30))
df_expuestos_dia["expuestos"] = df_expuestos_acum.groupby(["departamento"])["expuestos"].diff()
df_expuestos_dia = df_expuestos_dia.dropna()
df_expuestos_dia = df_expuestos_dia.append(df_expuestos_primer_dia)
df_expuestos_dia = df_expuestos_dia.sort_values(by=['fecha', 'departamento'])
df_expuestos_dia.tail(3)

,fecha,departamento,expuestos
727,2020-06-04,tolima,18.0
753,2020-06-04,valle del cauca,754.0
779,2020-06-04,vaupes,152.0


In [29]:
# Merge df_ird_constantes_dia con df_expuestos_dia
df_eird_constantes_dia = df_ird_constantes_dia.merge(
    df_expuestos_dia,
    how="inner",
    left_on=["departamento", "fecha"],
    right_on=["departamento", "fecha"],
)
df_eird_constantes_dia = df_eird_constantes_dia[
    [
        "fecha",
        "departamento",
        "expuestos",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_eird_constantes_dia.groupby(["fecha"])["departamento"].count().tail(30)
#df_eird_constantes_dia.tail(30)

fecha
2020-05-10    30
2020-05-11    30
2020-05-12    30
2020-05-13    30
2020-05-14    30
2020-05-15    30
2020-05-16    30
2020-05-17    30
2020-05-18    30
2020-05-19    30
2020-05-20    30
2020-05-21    30
2020-05-22    30
2020-05-23    30
2020-05-24    30
2020-05-25    30
2020-05-26    30
2020-05-27    30
2020-05-28    30
2020-05-29    30
2020-05-30    30
2020-05-31    30
2020-06-01    30
2020-06-02    30
2020-06-03    30
Name: departamento, dtype: int64

In [30]:
# Merge df_ird_constantes_acum con df_expuestos_acum
df_eird_constantes_acum = df_ird_constantes_acum.merge(
    df_expuestos_acum,
    how="inner",
    left_on=["fecha", "departamento"],
    right_on=["fecha", "departamento"],
)
df_eird_constantes_acum = df_eird_constantes_acum[
    [
        "fecha",
        "departamento",
        "expuestos",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_eird_constantes_acum.groupby(["fecha"])["departamento"].count().tail(30)
#df_eird_constantes_acum.tail(30)

fecha
2020-05-10    30
2020-05-11    30
2020-05-12    30
2020-05-13    30
2020-05-14    30
2020-05-15    30
2020-05-16    30
2020-05-17    30
2020-05-18    30
2020-05-19    30
2020-05-20    30
2020-05-21    30
2020-05-22    30
2020-05-23    30
2020-05-24    30
2020-05-25    30
2020-05-26    30
2020-05-27    30
2020-05-28    30
2020-05-29    30
2020-05-30    30
2020-05-31    30
2020-06-01    30
2020-06-02    30
2020-06-03    30
Name: departamento, dtype: int64

## Datos susceptibles


In [31]:
# Calculo susceptibles con df_eird_constantes_dia
df_seird_constantes_dia = copy.deepcopy(df_eird_constantes_dia)
df_seird_constantes_dia["susceptibles"] = df_seird_constantes_dia["poblacion_total"] - df_seird_constantes_dia["expuestos"] - df_seird_constantes_dia["infectados"] - df_seird_constantes_dia["recuperados"] - df_seird_constantes_dia["decesos"]
df_seird_constantes_dia = df_seird_constantes_dia[
    [
        "fecha",
        "departamento",
        "susceptibles",
        "expuestos",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_seird_constantes_dia.head(1)

,fecha,departamento,susceptibles,expuestos,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
0,2020-05-10,amazonas,77188.0,1457.0,94.0,83.0,8.0,2791,898.6,78830,0.719,25.42,328.0,11292.0,68.0,828.0,1680.0,448.0,2132.0,704.0,2636.0,2396.0,2408.0,2420.0,4320.0,5628.0,4252.0,1820.0,4476.0,0.0,4.0,116.0,332.0,516.0,7102.0,2540.0


In [32]:
# Calculo susceptibles con df_eird_constantes_acum
df_seird_constantes_acum = copy.deepcopy(df_eird_constantes_acum)
df_seird_constantes_acum["susceptibles"] = df_seird_constantes_acum["poblacion_total"] - df_seird_constantes_acum["expuestos"] - df_seird_constantes_acum["infectados"] - df_seird_constantes_acum["recuperados"] - df_seird_constantes_acum["decesos"]
df_seird_constantes_acum = df_seird_constantes_acum[
    [
        "fecha",
        "departamento",
        "susceptibles",
        "expuestos",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_seird_constantes_acum.head(1)

,fecha,departamento,susceptibles,expuestos,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
0,2020-05-10,amazonas,76339,1457.0,527.0,472.0,35.0,2791,898.6,78830,0.719,25.42,328.0,11292.0,68.0,828.0,1680.0,448.0,2132.0,704.0,2636.0,2396.0,2408.0,2420.0,4320.0,5628.0,4252.0,1820.0,4476.0,NaN,4.0,116.0,332.0,516.0,7102.0,2540.0


## Resultado final


In [33]:
# Total x departamento SEIRD + variables constantes
fecha_actual = datetime.now().date() - timedelta(days=2)
fecha_actual = fecha_actual.strftime("%Y-%m-%d")
df_expuestos = df_expuestos_acum[df_expuestos_acum['fecha'] == fecha_actual]

df_eird_constantes = df_ird_constantes.merge(
    df_expuestos[["departamento", "expuestos"]],
    how="inner",
    left_on=["departamento"],
    right_on=["departamento"],
)

df_seird_constantes = copy.deepcopy(df_eird_constantes)
df_seird_constantes["susceptibles"] = df_seird_constantes["poblacion_total"] - df_seird_constantes["expuestos"] - df_seird_constantes["infectados"] - df_seird_constantes["recuperados"] - df_seird_constantes["decesos"]
df_seird_constantes = df_seird_constantes[
    [
        "departamento",
        "susceptibles",
        "expuestos",
        "infectados",
        "recuperados",
        "decesos",
        "cantidad_mayores_65",
        "ipm",
        "poblacion_total",
        "personas_km2",
        "edad_promedio",
        'anomalias_congenitas',
        'codiciones_orales',
        'condiciones_derivadas_durante_el_periodo_perinatal',
        'condiciones_maternas',
        'condiciones_neuropsiquiatricas',
        'deficiencias_nutricionales',
        'desordenes_endocrinos',
        'diabetes_mellitus',
        'enfermedades_cardiovasculares',
        'enfermedades_de_la_piel',
        'enfermedades_de_los_organos_de_los_sentidos',
        'enfermedades_digestivas',
        'enfermedades_genitourinarias',
        'enfermedades_infecciosas_y_parasitarias',
        'enfermedades_musculo_esqueleticas',
        'enfermedades_respiratorias',
        'infecciones_respiratorias',
        'lesiones_de_intensionalidad_indeterminada',
        'lesiones_intensionales',
        'lesiones_no_intensionales',
        'neoplasias_malignas',
        'otras_neoplasias',
        'signos_y_sintomas_mal_definidos',
        'traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas',
    ]
]
df_seird_constantes.to_csv("seird_constantes.csv", index=False)
df_seird_constantes.head(3)

,departamento,susceptibles,expuestos,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
0,antioquia,6644210.0,44900.0,1328,533.0,6.0,576614.0,4698.3,6690977.0,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
1,arauca,269444.0,1261.0,2,1.0,0.0,13880.0,311.2,270708.0,11.366,29.092,2036.0,34944.0,1060.0,5640.0,10506.0,2932.0,9348.0,3274.0,17604.0,12780.0,22234.0,15142.0,32520.0,25708.0,21556.0,8108.0,24854.0,16.0,100.0,1074.0,1268.0,4222.0,48664.0,15424.0
2,atlantico,2513643.0,25567.0,5403,1379.0,146.0,196909.0,858.5,2546138.0,751.517,32.435,39482.0,490292.0,13216.0,92378.0,276326.0,63446.0,269384.0,94614.0,528492.0,379022.0,426452.0,413394.0,669408.0,598656.0,648180.0,261712.0,620138.0,668.0,1314.0,25066.0,27472.0,72974.0,1032222.0,312074.0


In [34]:
# Total x dia x departamento SEIRD + variables constantes
df_seird_constantes_dia.to_csv("seird_constantes_dia.csv", index=False)
df_seird_constantes_dia[df_seird_constantes_dia["departamento"] == "antioquia"].tail(3)

,fecha,departamento,susceptibles,expuestos,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
661,2020-06-01,antioquia,6690191.0,717.0,68.0,1.0,0.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
691,2020-06-02,antioquia,6689588.0,1360.0,29.0,0.0,0.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
721,2020-06-03,antioquia,6690145.0,772.0,60.0,0.0,0.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0


In [35]:
# Total x dia x departamento SEIRD (acumulado) + variables constantes
df_seird_constantes_acum.to_csv("seird_constantes_acum.csv", index=False)
df_seird_constantes_acum[df_seird_constantes_acum["departamento"] == "antioquia"].tail(3)

,fecha,departamento,susceptibles,expuestos,infectados,recuperados,decesos,cantidad_mayores_65,ipm,poblacion_total,personas_km2,edad_promedio,anomalias_congenitas,codiciones_orales,condiciones_derivadas_durante_el_periodo_perinatal,condiciones_maternas,condiciones_neuropsiquiatricas,deficiencias_nutricionales,desordenes_endocrinos,diabetes_mellitus,enfermedades_cardiovasculares,enfermedades_de_la_piel,enfermedades_de_los_organos_de_los_sentidos,enfermedades_digestivas,enfermedades_genitourinarias,enfermedades_infecciosas_y_parasitarias,enfermedades_musculo_esqueleticas,enfermedades_respiratorias,infecciones_respiratorias,lesiones_de_intensionalidad_indeterminada,lesiones_intensionales,lesiones_no_intensionales,neoplasias_malignas,otras_neoplasias,signos_y_sintomas_mal_definidos,traumatismos__envenenamientos_u_algunas_otras_consecuencias_de_causas_externas
661,2020-06-01,antioquia,6.6465e+06,42768.0,1171.0,533.0,6.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
691,2020-06-02,antioquia,6.64511e+06,44128.0,1200.0,533.0,6.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
721,2020-06-03,antioquia,6.64428e+06,44900.0,1260.0,533.0,6.0,576614,4698.3,6690977,105.184,34.628,77776.0,1441756.0,33730.0,222582.0,850400.0,67454.0,761556.0,288104.0,1548418.0,856284.0,1165008.0,915700.0,1431330.0,1214796.0,1699124.0,602088.0,1489768.0,366.0,3640.0,62904.0,142530.0,183648.0,2716784.0,938382.0
